In [2]:
import pandas as pd

# Load both feature dataframes
feats_df = pd.read_csv('../data/processed/feats_df.csv')
feats_df_old = pd.read_csv('../data/processed/feats_df_old.csv')

print("New features dataframe shape:", feats_df.shape)
print("Old features dataframe shape:", feats_df_old.shape)


New features dataframe shape: (286, 20)
Old features dataframe shape: (286, 18)


In [3]:
# Check column differences
print("Columns in new version:", list(feats_df.columns))
print("\nColumns in old version:", list(feats_df_old.columns))

# Find columns only in new version
new_cols = set(feats_df.columns) - set(feats_df_old.columns)
print(f"\n✨ New columns added: {new_cols if new_cols else 'None'}")

# Find columns only in old version
removed_cols = set(feats_df_old.columns) - set(feats_df.columns)
print(f"❌ Columns removed: {removed_cols if removed_cols else 'None'}")


Columns in new version: ['id', 'Q1_CREATIVE', 'Q2_CREATIVE', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'error', 'Q1_SBL', 'Q2_SBL']

Columns in old version: ['id', 'Q1_CREATIVE', 'Q2_CREATIVE', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'error', 'Q1_SBL', 'Q2_SBL']

✨ New columns added: {'Q16', 'Q15'}
❌ Columns removed: None


In [4]:
# Compare row counts and check for missing/new IDs
print(f"Number of rows:")
print(f"  New version: {len(feats_df)}")
print(f"  Old version: {len(feats_df_old)}")
print(f"  Difference: {len(feats_df) - len(feats_df_old)}")

# Check IDs
new_ids = set(feats_df['id'])
old_ids = set(feats_df_old['id'])

ids_added = new_ids - old_ids
ids_removed = old_ids - new_ids

print(f"\n📊 IDs added in new version: {len(ids_added)}")
if len(ids_added) > 0 and len(ids_added) <= 10:
    print(f"   {ids_added}")

print(f"📊 IDs removed from old version: {len(ids_removed)}")
if len(ids_removed) > 0 and len(ids_removed) <= 10:
    print(f"   {ids_removed}")


Number of rows:
  New version: 286
  Old version: 286
  Difference: 0

📊 IDs added in new version: 0
📊 IDs removed from old version: 0


In [5]:
# Check for errors in both versions
print("Error analysis:")
print(f"  New version - rows with errors: {feats_df['error'].notna().sum()}")
print(f"  Old version - rows with errors: {feats_df_old['error'].notna().sum()}")

# Show sample of errors if any
if feats_df['error'].notna().sum() > 0:
    print("\nSample errors in new version:")
    print(feats_df[feats_df['error'].notna()][['id', 'error']].head())


Error analysis:
  New version - rows with errors: 4
  Old version - rows with errors: 5

Sample errors in new version:
                                       id  \
12   07502100-4f0c-4917-b59c-067c80f1476f   
198  4a7bbaff-0e2c-4467-982e-086f93b71f63   
201  aafc0685-c1d1-48a9-80cc-af89d93bc836   
202  91137d04-8dc4-4f11-bd81-1b4090a5766c   

                                         error  
12   Expecting value: line 1 column 1 (char 0)  
198  Expecting value: line 1 column 1 (char 0)  
201  Expecting value: line 1 column 1 (char 0)  
202  Expecting value: line 1 column 1 (char 0)  


In [6]:
# Compare values for common columns on common IDs
common_ids = new_ids & old_ids
common_cols = set(feats_df.columns) & set(feats_df_old.columns)
common_cols.discard('id')  # Don't compare id with itself

print(f"\nComparing {len(common_ids)} common IDs across {len(common_cols)} common columns...")

# Merge on id to compare
comparison = feats_df[feats_df['id'].isin(common_ids)].merge(
    feats_df_old[feats_df_old['id'].isin(common_ids)],
    on='id',
    suffixes=('_new', '_old')
)

print(f"Merged dataframe shape: {comparison.shape}")



Comparing 286 common IDs across 17 common columns...
Merged dataframe shape: (286, 37)


In [7]:
# Find differences in each common column
differences_summary = {}

for col in common_cols:
    if col in ['error']:  # Skip error column for now
        continue
    
    col_new = f"{col}_new"
    col_old = f"{col}_old"
    
    # Handle NaN values - consider them equal
    differences = comparison[
        (comparison[col_new].fillna('') != comparison[col_old].fillna('')) &
        ~(comparison[col_new].isna() & comparison[col_old].isna())
    ]
    
    num_diffs = len(differences)
    if num_diffs > 0:
        differences_summary[col] = num_diffs

# Display summary
print("\n📋 Column-wise differences summary:")
print("=" * 60)
for col, count in sorted(differences_summary.items(), key=lambda x: x[1], reverse=True):
    print(f"  {col:25s}: {count:4d} changes ({count/len(common_ids)*100:.1f}%)")

print(f"\nTotal columns with changes: {len(differences_summary)}")



📋 Column-wise differences summary:
  Q14                      :  104 changes (36.4%)
  Q6                       :   58 changes (20.3%)
  Q4                       :   45 changes (15.7%)
  Q10                      :   45 changes (15.7%)
  Q3                       :   43 changes (15.0%)
  Q2_CREATIVE              :   38 changes (13.3%)
  Q8                       :   36 changes (12.6%)
  Q12                      :   33 changes (11.5%)
  Q9                       :   32 changes (11.2%)
  Q1_SBL                   :   24 changes (8.4%)
  Q5                       :   19 changes (6.6%)
  Q1_CREATIVE              :   12 changes (4.2%)
  Q13                      :   12 changes (4.2%)
  Q7                       :    3 changes (1.0%)
  Q11                      :    3 changes (1.0%)

Total columns with changes: 15


In [8]:
# Show detailed examples of changes for columns with most differences
print("\n🔍 Sample changes for top columns with differences:\n")

for col in list(sorted(differences_summary.items(), key=lambda x: x[1], reverse=True))[:5]:
    col_name = col[0]
    col_new = f"{col_name}_new"
    col_old = f"{col_name}_old"
    
    # Find rows where this column changed
    changed_rows = comparison[
        (comparison[col_new].fillna('') != comparison[col_old].fillna('')) &
        ~(comparison[col_new].isna() & comparison[col_old].isna())
    ]
    
    print(f"\n{col_name} ({differences_summary[col_name]} changes):")
    print("-" * 80)
    
    # Show first 3 examples
    for idx, row in changed_rows.head(3).iterrows():
        print(f"  ID: {row['id']}")
        print(f"    Old: {row[col_old]}")
        print(f"    New: {row[col_new]}")
        print()



🔍 Sample changes for top columns with differences:


Q14 (104 changes):
--------------------------------------------------------------------------------
  ID: a43a50b5-5177-456e-b2dd-ea45272e3cd2
    Old: Unknown
    New: Informational

  ID: 87023e02-5eb3-4257-937f-079a8902d53a
    Old: Unknown
    New: Informational

  ID: ff1592ba-8297-4a6f-bf29-105456db6ca3
    Old: Headline
    New: Tagline


Q6 (58 changes):
--------------------------------------------------------------------------------
  ID: 2889f0ae-360b-4419-b304-49da3283b22e
    Old: Studio
    New: Neutral Textural

  ID: 94802d11-dd96-46d7-840a-ed946019f3fb
    Old: Studio
    New: Natural Light / Lifestyle

  ID: bbbb666c-e925-485c-8c5b-0de8b426abb1
    Old: Everyday Lifestyle
    New: Bedroom / Home


Q4 (45 changes):
--------------------------------------------------------------------------------
  ID: a43a50b5-5177-456e-b2dd-ea45272e3cd2
    Old: Comfort & Softness
    New: Fit & Support

  ID: 78d732c7-ca8a-47c9-b4ef

In [ ]:
# Check for value distribution changes in key categorical columns
categorical_cols = ['Q1_CREATIVE', 'Q2_CREATIVE', 'Q3', 'Q4', 'Q5', 'Q1_SBL', 'Q2_SBL']

print("\n📊 Value distribution comparison for key columns:\n")

for col in categorical_cols:
    if col not in common_cols:
        continue
    
    print(f"\n{col}:")
    print("=" * 80)
    
    # Get value counts for both versions
    new_counts = feats_df[col].value_counts()
    old_counts = feats_df_old[col].value_counts()
    
    # Combine and compare
    all_values = set(new_counts.index) | set(old_counts.index)
    
    comparison_df = pd.DataFrame({
        'New': [new_counts.get(v, 0) for v in all_values],
        'Old': [old_counts.get(v, 0) for v in all_values],
    }, index=list(all_values))
    
    comparison_df['Diff'] = comparison_df['New'] - comparison_df['Old']
    comparison_df = comparison_df.sort_values('New', ascending=False)
    
    # Only show if there are differences
    if comparison_df['Diff'].abs().sum() > 0:
        print(comparison_df.head(10))
    else:
        print("  No distribution changes")


In [ ]:
# Summary statistics
print("\n" + "=" * 80)
print("SUMMARY")
print("=" * 80)
print(f"\nDataset size:")
print(f"  Old: {len(feats_df_old)} rows, {len(feats_df_old.columns)} columns")
print(f"  New: {len(feats_df)} rows, {len(feats_df.columns)} columns")
print(f"  Change: {len(feats_df) - len(feats_df_old):+d} rows, {len(feats_df.columns) - len(feats_df_old.columns):+d} columns")

print(f"\nData quality:")
print(f"  Old: {feats_df_old['error'].notna().sum()} rows with errors ({feats_df_old['error'].notna().sum()/len(feats_df_old)*100:.1f}%)")
print(f"  New: {feats_df['error'].notna().sum()} rows with errors ({feats_df['error'].notna().sum()/len(feats_df)*100:.1f}%)")

if len(differences_summary) > 0:
    print(f"\nChanges in existing data:")
    print(f"  {len(differences_summary)} columns have value changes")
    print(f"  {len(common_ids)} common IDs between versions")
    
    # Calculate total number of cell changes
    total_changes = sum(differences_summary.values())
    total_cells = len(common_ids) * len(common_cols)
    print(f"  {total_changes} total value changes across {total_cells} comparable cells ({total_changes/total_cells*100:.2f}%)")

print(f"\nNew columns: {list(new_cols) if new_cols else 'None'}")
print(f"Removed columns: {list(removed_cols) if removed_cols else 'None'}")
